# Practical session 2: Implementing reactive behaviors

## Preliminary notes

**Reminders :** 
- Save your work as indicated at the beginning of the `session_1.ipynb` notebook.
- Each time you encounter a cell containing code in this notebook (as in the cell starting with `from vivarium.controllers...` below), you have to execute it by clicking on the cell and pressing `Shift+Enter` (unless it is explicitly specified not to do it). This will import the necessary modules and functions to use the simulator.

<!-- TODO: load scene !-->

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Then, launch the simulator for this session as well as the interface : 

In [2]:
start_server_and_interface(scene_name="session_2")

/home/cleger/Desktop/code/vivarium/vivarium/utils

STARTING SERVER


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[2024-12-04 14:50:02,187][__main__][INFO] - Scene running: session_2
[2024-12-04 14:50:04,633][vivarium.simulator.simulator][INFO] - Simulator initialized

STARTING INTERFACE


2024-12-04 14:50:07,563 Starting Bokeh server version 3.3.4 (running on Tornado 6.4)
2024-12-04 14:50:07,564 User authentication hooks NOT provided (default user enabled)
2024-12-04 14:50:07,565 Bokeh app running at: http://localhost:5006/run_interface
2024-12-04 14:50:07,565 Starting Bokeh server with process id: 12728


Wait until the interface link shows up (http://localhost:5006/run_interface) and click on it, and make sure the scene is present on your browser. Finally, create a controller that will be used to control the simulation with Python code from this jupyter notebook.

In [3]:
controller = NotebookController()

/home/cleger/Desktop/code/vivarium/venv/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Then start the controller with the following code:

In [4]:
controller.run()

Simulator is already stopped


### COMMENT : Think this should not be here, but we should put it in another notebook

If at one point things are not going as expected (e.g. the agent doesn't move the way it should in the simulator), do the following steps:

- Check if the controller is still running:  

In [7]:
controller.is_running()

True

- If it is not running, restart it with:  

In [8]:
controller.run()

Simulator is already started


- Else, check if it works by making an agent turn on itself for example: 

In [9]:
controller.agents[0].left_motor = 0.5

If this doesn't work, it means there is either a problem with the controller or the simulator. In this case, simply restart both with the following steps:

- Stop any code that can still be executing by pressing the "stop-like" or "restart" button in the top menu bar of this document. 
- Stop the simulator by executing (no need to do it now though):

In [10]:
controller.stop()

- Restart the notebook by clicking `Kernel -> Restart` in the menu.
- Re-open the session by executing (don't re-open it now if you haven't closed it by executing the previous cell):

In [11]:
from vivarium.controllers.notebook_controller import NotebookController
controller = NotebookController()

In [12]:
controller.run()

We can create an alias for the agent again since there is only one:

In [13]:
controller.agents

In [14]:
agent = controller.agents[0]

## Behaviors

In the last practical session we saw how to set the agent left and right motor speeds as well as how to read the values returned by the left and right proximeters. We programmed a first simple behavior where the agent slows down when approaching an obstacle. 

Here is a possible solution for this behavior, if will run for 20 seconds if you execute the next cell:

In [15]:
# Repeat 200 times the indented code:
for i in range(200):
    # print the iteration number every 20 iterations
    if i % 20 == 0:
        print(f"Iteration {i}")
        
    # Read the proximeter values and store them in the "left" and "right" variables
    left, right = agent.sensors()
    
    # Compute the sum of the values returned by the left and right proximeters.
    # This sum will be between 0 and 2 because both "left" and "right" are between 0 and 1
    sum_of_proxs = left + right
    
    # Compute the activation that will be applied to both motors. 
    # The closer the obstacle (i.e. the higher the value returned by the proximeters), the lower the motor activation should be.
    # Note that motor activation is bounded between 0 and 1
    motor_activation = 1.0 - sum_of_proxs / 2.0
    
    # Set the activation of both motors to the value we just have computed
    agent.right_motor = motor_activation
    agent.left_motor = motor_activation
    
    # Waits for 100 milliseconds before starting the next iteration (to avoid overloading you computer)
    controller.wait(0.1)

agent.right_motor = agent.left_motor = 0.0

Iteration 0
Iteration 20
Iteration 40
Iteration 60


2024-12-04 14:51:42,316 An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
2024-12-04 14:51:44,010 WebSocket connection opened
2024-12-04 14:51:44,028 ServerConnection created


Iteration 80
Iteration 100
Iteration 120
Iteration 140
Iteration 160
Iteration 180


## Practical definition of a behavior

The example behavior defined above illustrates the general structure of a behavior. 

**Definition:** a behavior consists of a loop repeated at a certain frequency where (1) the values of relevant sensors are read, (2) some computation is performed using these values and (3) commands are sent to the agent motors according to the result of this computation.

Step (1) corresponds to the reading of the left and right proximeters activations. Step (2) corresponds to the computation of `motor_activation` according to the sum of the proximeter activations. Finally, Step (3) corresponds to setting the speed of both motors to the value of `motor_activation`.

Note that the code above will take a while to be executed (approximately `200 * 0.1 = 20` seconds, since the loop is repeated 200 times). During this time, you can't execute anything else in this notebook. To stop the execution before it terminates by itself, you have to press the "stop-like" button in the top menu bar of this document. 

This approach has three major drawbacks:
- Only one behavior can run at a time.
- The behavior has a fixed duration (at one point it will stop)
- We can't stop a behavior programmatically (instead we have to press the "stop-like" button).

To overcome these problems, we provide a more flexible method for defining and executing behaviors. Let's rewrite the behavior above using that method. First make sure the previous code is not still being executed by pressing the "stop-like" button in the top menu bar of this document. Now, defining a behavior boils down to defining a function which includes the core of the behavioral loop:

In [16]:
# The code in this cell defines a function called slow_down (first line),
# which takes as argument the agent (first line, in parenthesis),
# and returns the left and right wheel activation to be applied to the motors (last line)

def slow_down(agent):
    # Step (1): read the sensor values
    left, right = agent.sensors()
    
    # Step (2): do some computation
    sum_of_proxs = left + right
    motor_activation = 1.0 - sum_of_proxs / 2.0
    
    # Step (3): return the motor activations for left and right motors
    return motor_activation, motor_activation

The cell above defines a function called `slow_down`. In computer programming, a function is a sequence of instructions that perform a specific task depending on some parameters (called the arguments of the function) and that returns a result. In this sense it is very similar to the mathematical definition of a function, as for example when we write `y = f(x)`, where `f` is the name of the function, `x` is its argument, and `y` is the result. For example, we can define a function `square` that computes the square of its argument as follows:


In [17]:
def square(x):
    return x * x

print(square(3))

9



As seen above, the definition of a function in Python starts with the keyword `def`, followed by the arbitrary name we choose for the function (here we called it `slow_down` to reflect the purpose of the behavior defined in it). Then come the arguments of the function in parenthesis (in our case it will be the variable representing the agent, called `agent`) and finally the symbol `:`. Below the first line, you find the instructions that this function will execute when it will be called. Those instructions need to be intended right with respect to the first line. In this example, the instructions are the exact same as in the core of the previous `for` loop, except that:
- we omit the last line `controller.wait(0.1)` (the frequency at which the behavior will be executed will be set in more rigorous way below),
- we don't directly set the motor activations using `agent.left_motor` and `agent.right_motor`. Instead, we *return* the values of the motor activations in the last line and they will be automatically sent to the agent motors when the behavior will be executed. In the last line, the values after the `return` keyword have to be the left and right wheel activation (in this order). Both activations have to be between 0 and 1. (In the `slow_down` behavior above, both activations are the same since we don't want the agent to turn).

Note that a function definition, as the one above, does not execute the instructions contained in it, it only defines them so that they can be executed later when the function will be *called*. In our case, we will not explicitly call the function, instead it will be done behind the scene when we will start the behavior on the agent (see below).

Once the behavior is defined as a function, we can attach it to the agent by executing:


In [18]:
agent.attach_behavior(slow_down)

The line above means: attach the behavior defined in the function `slow_down` to the `agent`. By default, this instruction also executes the behavior on the robot. If you don't want to start the behavior when attaching it for some reason, you can set the optional argument `start` to `False` as follows:

In [19]:
agent.attach_behavior(slow_down, start=False)

Note that if you executed the cell above, the behavior will still be executed on the agent, because it was already started in the cell before. We can check it with the following command: 

In [20]:
agent.print_behaviors()

Available behaviors: ['slow_down'], Active behaviors: ['slow_down']


### Comment: see if we agree on the API before modifying this

You should now see the agent executing the exact same behavior as before (if the agent is already close to an obstacle, move it to a more open space to observe it slowing down).

The line above means: start running the previously attached `slow_down` behavior on the `agent`. Executing the above line will basically do the same thing as executing the `for` loop at the start of this document. Here, the function `slow_down` will be executed indefinitely. 

Using this method has the following advantages over the previous method using the `for` loop:
- It is more compact to write and it will allow to better structure your code when you will have to deal with multiple behaviors and multiple agents.
- It is not blocking as the previous method was. This means that you can still use this notebook while the behavior is running on the agent. For example, let's read the proximeter activations while the agent is still executing the `slow_down` behavior:

Each time you execute the cell above, you should see the proximeter activation changing because the agent is moving. However, you should avoid setting motor values while a behavior is running since this could conflict with the behavior also setting those values. When a behavior is started, it runs indefinitely until you explicitly tell it to stop. To do so, you have to execute:

In [21]:
agent.stop_behavior("slow_down")

Note that the agent will continue moving using the last motor speeds that were set by the behavior. You can set both motor speeds to 0 by executing:

In [22]:
agent.stop_motors()

Then start the behavior again by executing:

In [23]:
agent.start_behavior("slow_down")

You can also stop the behavior and stop the motors of the agent in the same line, by using the `stop_motors` argument in the `detach_behavior` function:

In [23]:
agent.detach_behavior("slow_down", stop_motors=True)

The stop_motors argument will automatically execute agent.stop_motors() before detaching the behavior. If you don't want the motors to stop when detaching the behavior, you can set stop_motors to False.

At anytime, you can check what behaviors are attached to the agent with:

In [24]:
agent.print_behaviors()

Available behaviors: ['slow_down'], Active behaviors: ['slow_down']


By default, the behaviors of the agents will be running every 5 steps of the simulation, which can slow down the simulation. You can actually choose an interval of execution for each behavior by setting the `interval` argument when attaching the behavior to the agent. For example, to execute the `slow_down` behavior every 10 steps of the simulation, you can execute:

In [26]:
# detach the current behavior
agent.detach_behavior("slow_down")
# attach the new one with a different interval
agent.attach_behavior(slow_down, interval=10)
agent.start_behavior("slow_down")
agent.print_behaviors()

Available behaviors: ['slow_down'], Active behaviors: ['slow_down']


To fix a good execution interval for the behavior, you can check the fps (steps per second) of the simulation by executing:

In [27]:
controller.print_fps()

measuring the FPS (number of steps per second) in the controller during 2 seconds ...


FPS: 51.00


### COMMENT : Do we introduce the detach_all_behaviors function here or in a later notebook ?

In [28]:
agent.detach_all_behaviors(stop_motors=True)

**Q1:** To make sure you correctly understand how to attach, start, stop and detach a behavior, complete the following code:

In [29]:
# First we make sure that no behavior is attach to the agent:
agent.detach_all_behaviors()

# When checking, it should print "No behavior attached":
print("\nstep 1:")
agent.print_behaviors() # This will print "No behavior attached"

# Write just below this line the code that attaches the slow_down behavior
agent.attach_behavior(slow_down)
print("\nstep 2:")
agent.print_behaviors() # This will print "Available behaviors: ['slow_down'], Active behaviors: No active behaviors"

# Write just below this line the code that starts the slow_down behavior
agent.start_behavior("slow_down")
print("\nstep 3:")
agent.print_behaviors() # This will print "Available behaviors: ['slow_down'], Active behaviors: ['slow_down']"

# Write just below this line the code that stops the slow_down behavior
agent.stop_behavior("slow_down")
print("\nstep 4:")
agent.print_behaviors() # This will print "Available behaviors: ['slow_down'], Active behaviors: No active behaviors"

# Write just below this line the code that detaches the slow_down behavior
agent.detach_behavior("slow_down")
print("\nstep 5:")
agent.print_behaviors() # This will print "No behavior attached"


step 1:
No behaviors attached

step 2:
Available behaviors: ['slow_down'], Active behaviors: ['slow_down']

step 3:
Available behaviors: ['slow_down'], Active behaviors: ['slow_down']

step 4:
Available behaviors: ['slow_down'], Active behaviors: No active behaviors

step 5:
No behaviors attached


Let's summarize the method we have just describe to define, attach, start, stop and detach a behavior:

In [30]:
# First, detach all the behaviors that might still be attached to the agent
# (it is a good practice to do it each time you want to define a new behavior, or modify an existing one):
agent.detach_all_behaviors()

In [31]:
# Define a behavior where the agent progressively slows down when it approaches an obstacle:
def slow_down(agent):
    # Step (1): read the sensor values
    left, right = agent.sensors()
    
    # Step (2): do some computation
    sum_of_proxs = left + right
    motor_activation = (2 - sum_of_proxs) / 2
    
    # Step (3): return the motor activations
    return motor_activation, motor_activation

In [33]:
# Attach and start this behavior to the agent, and specify the step interval at which it will be executed
agent.attach_behavior(slow_down, interval=10)

When executing the code above, you should see the behavior being executed on the agent in the simulator. Then, to stop and detach the behavior:

In [34]:
agent.stop_behavior(slow_down)
agent.detach_behavior(slow_down, stop_motors=True)

An alternative way is to stop and detach all the behaviors running on the agent. This avoids having to specify the name of the behavior (`slown_down` in the cell above) and also stops systematically the behavior before detaching it:

+ stop motors (should me make it a default arg with this function ? )

In [35]:
agent.detach_all_behaviors(stop_motors=True)

## Implementing the Braitenberg vehicles

Let's now practice a bit. Remember the Braitenberg Vehicle examples we have seen in [this slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0). Those vehicles are very similar to the agents in the simulator. 
- It is equipped with two sensors that are activated according to the proximity of a source. With the agent, each proximeter sensor returns a value between 0 and 1 that is inversely proportional to the distance from the closest obstacle it perceives (the closer the obstacle, the highest to proximeter activation).
- It is equipped with two motors allowing the agent to move. With the agent, we can set the activation of each motor independently with a value between 0 and 1 (where 1 means maximum speed). 
- A behavior links sensor activations to motor activations. In the Braitenberg vehicles, this is achieved through connections that are either excitatory (the activity of the sensor increases the activity of the motor it is connected to) or inhibitory (the activity of the sensor decreases the activity of the motor it is connected to). In the agent, we have seen above that we can define a behavior as a function that (1) read the sensor activities (2) perform some computation and (3) use the result of that computation to set the motor speed. 

Therefore, we can implement in the agent the various types of vehicle behaviors shown in the slide, where defining excitatory and inhibitory connections will be done through Step (2) above (*perform some computation*). We have actually already done it with the `slow_down` behavior we have defined above.  

**Q2:** Define verbally the `slow_down` behavior in term of inhibitory and excitatory connections (do it by double clicking on the next cell). Your answer must look like this (where `TO_FILL` is either the word "excitatory" or "inhibitory"):
- The activity of the left sensor is connected to the left motor through a TO_FILL connection.
- The activity of the left sensor is connected to the right motor through a TO_FILL connection.
- The activity of the right sensor is connected to the left motor through a TO_FILL connection.
- The activity of the right sensor is connected to the right motor through a TO_FILL connection.

*Double click on this cell and replace this text by your answer*

Let's see how to define the `fear` behavior illustrated in [the slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0) using the method we have seen: 

In [36]:
def fear(agent):
    left, right = agent.sensors()
    return left, right

That's pretty easy, isn't it? As illustrated in the slide, the `fear` behavior simply consists in the left sensor exciting the left motor, and the right sensor exciting the right motor. Therefore, the simplest way of programming this behavior is to directly map the left and right sensor activations to the left and right motor speed, respectively. This is what is done in the function definition just above.

Let's now analyze the properties of this `fear` behavior in more detail. Attach and start the `fear` behavior by executing the cell below, and observe how the agent behaves.

In [37]:
agent.detach_all_behaviors()  # Just in case a behavior is still attached

agent.attach_behavior(fear)
agent.start_all_behaviors()

In [38]:
agent.sensors()

[0.5557094812393188, -0.0]

In [39]:
agent.detach_all_behaviors(stop_motors=True)

In [40]:
agent.print_infos()

Entity Overview:
--------------------
Type: AGENT
Subtype: AGENTS
Idx: 0
Exists: True
Position: x=180.33, y=31.01
Diameter: 10.00
Color: #0000ff

Sensors: Left=0.19, Right=-0.00
Motors: Left=0.00, Right=0.00



**Q3:** Use the cell below to answer the following questions.
<!-- TODO: change that because there is no walls/corners !-->
1. What happens when the activity of both sensors is null? (i.e. no obstacle is detected.) Why?
2. How does the agent react when it detects an obstacle? (e.g. an object.) Why?
3. Imagine a small animal equipped with such a behavior in the wild. What would be its evolutionary advantages and drawbacks? (could it escape from a predator? could it collect food? Could it hide itself?)

*Double click on this cell and replace this text by your answer*

**Q4:** Program the `aggression` behavior illustrated in [the slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0), which consists of crossed excitatory connections. 

In [41]:
def aggression(agent):
    left, right = agent.sensors()
    return right, left

Before executing the behavior you have defined in the cell just above, first detach the previous one and immobilize the agent:

In [42]:
agent.detach_all_behaviors(stop_motors=True)

Then attach the `aggression` behavior and start it:

In [43]:
agent.attach_behavior(aggression)
agent.start_all_behaviors()

**Q5:** Use the cell below to answer the following questions.

3. How does the agent reacts when it approaches an object?  Why?
2. How does the agent react when close to a moveable object (the orange obstacles in the scene) Why?
4. Imagine an animal equipped with such a behavior in the wild. What would be its evolutionary advantages and drawbacks? (could it escape from a predator? could it catch preys? Could it hide itself? Could it move things?)

*Double click on this cell and replace this text by your answer*

That's it for this practical session. You can now close the session:

In [45]:
controller.stop()
stop_server_and_interface(safe_mode=False)

 Found the process scripts/run_interface.py running with this PID: 12728
 Found the process scripts/run_server.py running with this PID: 12487
Killed process with PID: 12728
Killed process with PID: 12487

Server and Interface processes have been stopped



Simulator is already stopped

Stopping server and interface processes

Received signal 15, shutting down


False

Now that you finished session 2, you can now jump to the notebook of [session 3](session_3.ipynb).